In [3]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from imgaug import augmenters as iaa
from tqdm import tqdm_notebook as tqdm
import json
import copy
import mmcv
from multiprocessing import Pool
import sys
import skimage.io as io
from pycocotools.coco import COCO   # 载入 cocoz
%matplotlib inline
import random
from PIL import Image
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

In [ ]:
def get_info(fp):
    masks = []
    labels = []
    boxes = []
    for line in fp:
        line = line.strip()
        if not line:
            continue
        items = line.split(' ')
        loc = [float(x) for x in items[:8]]
        xs = loc[0::2]
        ys = loc[1::2]
        poly = np.array([[[loc[0],loc[1]],[loc[2],loc[3]],[loc[4],loc[5]],[loc[6], loc[7]]]], np.int32)
        area = cv2.contourArea(poly)
        xmin  = min(xs)
        xmax = max(xs)
        ymin = min(ys)
        ymax = max(ys)
        w = xmax - xmin
        h = ymax - ymin
        cls = items[8]
        if cls not in CLASS:
            continue
        labels.append(cls)
        masks.append(loc)
        boxes.append([xmin,ymin,xmax,ymax])
    return labels, masks, boxes

def gen_ann(box, mask, label, annotations, obj_id, ID, xmin, ymin):
    for b, loc, l in zip(box, mask, label):
        x1, y1, x2, y2 = b
        x1 -= xmin
        y1 -= ymin
        x2 -= xmin
        y2 -= ymin
        poly = np.array([[[loc[0],loc[1]],[loc[2],loc[3]],[loc[4],loc[5]],[loc[6], loc[7]]]], np.int32)
        box = [x1, y1, x2, y2]
        area = cv2.contourArea(poly)
        poly = [loc[0]-xmin,loc[1]-ymin,loc[2]-xmin,loc[3]-ymin,loc[4]-xmin,loc[5]-ymin,loc[6]-xmin, loc[7]-ymin]
        annotation = {"id" : obj_id, 
                "image_id" : ID, 
                "category_id" : class_to_ind[l], 
                "segmentation" : [poly], 
               "area" :  area, 
               "bbox" : [x1, y1, x2-x1, y2-y1], 
               "iscrowd" : 0,
            }
        obj_id += 1
        annotations["annotations"].append(annotation)
    ID += 1
    return annotations, obj_id, ID
  #         poly = np.array([[[loc[0],loc[1]],[loc[2],loc[3]],[loc[4],loc[5]],[loc[6], loc[7]]]], np.int32)
            #         img2 = cv2.polylines(img2, poly, 1,(0,255,0), 1)
                    #cv2.rectangle(img2 , (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)
            #         poly = np.array([[[loc[0]-xmin,loc[1]-ymin],[loc[2]-xmin,loc[3]-ymin],[loc[4]-xmin,loc[5]-ymin],[loc[6]-xmin, loc[7]-ymin]]], np.int32)
            #         cv2.polylines(img2, poly, 1,(0,255,0), 2)
            #         cv2.fillPoly(img2, poly, 255)

In [4]:
def bbox_overlaps_py(boxes, query_boxes):
    """
    determine overlaps between boxes and query_boxes
    :param boxes: n * 4 bounding boxes
    :param query_boxes: k * 4 bounding boxes
    :return: overlaps: n * k overlaps
    """
    n_ = boxes.shape[0]
    k_ = query_boxes.shape[0]
    overlaps = np.zeros((n_, k_), dtype=np.float)
    for k in range(k_):
        query_box_area = (query_boxes[k, 2] - query_boxes[k, 0] + 1) * (query_boxes[k, 3] - query_boxes[k, 1] + 1)
        for n in range(n_):
            iw = min(boxes[n, 2], query_boxes[k, 2]) - max(boxes[n, 0], query_boxes[k, 0]) + 1
            if iw > 0:
                ih = min(boxes[n, 3], query_boxes[k, 3]) - max(boxes[n, 1], query_boxes[k, 1]) + 1
                if ih > 0:
                    box_area = (boxes[n, 2] - boxes[n, 0] + 1) * (boxes[n, 3] - boxes[n, 1] + 1)
                    all_area = float(query_box_area)
                    overlaps[n, k] = iw * ih / all_area
    return overlaps

def _pygenerate(boxes, masks, labels, width, height, chipsize, stride):
        chips = []
        boxes = np.array(boxes)
        masks = np.array(masks)
        # ensure coverage of image for worst case
        # corners
        chips.append([max(width - chipsize, 0), 0, width - 1, min(chipsize, height-1)])
        chips.append([0, max(height - chipsize, 0), min(chipsize, width-1), height-1])
        chips.append([max(width - chipsize, 0), max(height - chipsize, 0), width-1, height-1])
		
        for i in range(0, width - int(chipsize), stride):
            for j in range(0, height - int(chipsize), stride):
                x1 = i
                y1 = j
                x2 = i + chipsize - 1
                y2 = j + chipsize - 1
                chips.append([x1, y1, x2, y2])
		#width may not be divide by stride
        for j in range(0, height - int(chipsize), stride):
            x1 = max(width - chipsize - 1,0)
            y1 = j
            x2 = width - 1
            y2 = j + chipsize - 1
            chips.append([x1, y1, x2, y2])
		#the same as above
        for i in range(0, width - int(chipsize), stride):
            x1 = i
            y1 = max(height - chipsize - 1,0)
            x2 = i + chipsize - 1
            y2 = height - 1
            chips.append([x1, y1, x2, y2])

        chips = np.array(chips).astype(np.float)
        overlaps = bbox_overlaps_py(chips, boxes.astype(np.float))
        fchips = []
        masks_list = []
        boxes_list = []
        labels_list = []
        for j in range(len(chips)):
            nvids = np.where(overlaps[j, :] >= 0.9)[0]
            if(len(nvids) == 0):
                continue
            else:
                fchips.append(chips[j])
                boxes_list.append(boxes[nvids])
                masks_list.append(masks[nvids])
                labels_list.append([labels[x] for x in nvids])
        return fchips, masks_list, boxes_list, labels_list

def test_pygenerate(width, height, chipsize, stride):
        chips = []
        # ensure coverage of image for worst case
        # corners
        chips.append([max(width - chipsize, 0), 0, width - 1, min(chipsize, height-1)])
        chips.append([0, max(height - chipsize, 0), min(chipsize, width-1), height-1])
        chips.append([max(width - chipsize, 0), max(height - chipsize, 0), width-1, height-1])
		
        for i in range(0, width - int(chipsize), stride):
            for j in range(0, height - int(chipsize), stride):
                x1 = i
                y1 = j
                x2 = i + chipsize - 1
                y2 = j + chipsize - 1
                chips.append([x1, y1, x2, y2])
		#width may not be divide by stride
        for j in range(0, height - int(chipsize), stride):
            x1 = max(width - chipsize - 1,0)
            y1 = j
            x2 = width - 1
            y2 = j + chipsize - 1
            chips.append([x1, y1, x2, y2])
		#the same as above
        for i in range(0, width - int(chipsize), stride):
            x1 = i
            y1 = max(height - chipsize - 1,0)
            x2 = i + chipsize - 1
            y2 = height - 1
            chips.append([x1, y1, x2, y2])

        chips = np.array(chips).astype(np.float)
        return chips
def generate_test(datdir, phase):
    info = {
        "description": "rscup",
        "url": "http://cocodataset.org",
        "version": "1.0",
        "year": 2014,
        "contributor": "COCO Consortium",
        "date_created": "2017/09/01"
      }
    license = [{ "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/", "id": 1, "name": "Attribution-NonCommercial-ShareAlike License"} ]
    categories = []
    for cls in CLASS:
        category = { "id" : class_to_ind[cls], "name" : cls, "supercategory" : "object",}
        categories.append(category)

    annotations = {"info": info, "images": [], "annotations": [], "categories":categories, "license":license}
    imgs = os.listdir(datadir)
    ID = 0
    for img_id in tqdm(range(len(imgs))):
        img_path = os.path.join(datadir, imgs[img_id])
        img = cv2.imread(img_path)
        H,W,_ = img.shape
        if max(H,W) > 512:
            fchips = test_pygenerate(W, H, 512, 416)
            count = 0
            for chip in fchips:
                count+=1
                xmin, ymin, xmax, ymax = chip
                filename = imgs[img_id].split(".")[0] + "_{}_{}_".format(str(xmin), str(ymin))+"_part" +str(count) + ".jpg"
                image = { "license": 1,
                          "file_name": filename,
                          "coco_url": "xxx",
                          "height": ymax-ymin,
                          "width": xmax-xmin,
                          "date_captured": "2019-06-25",
                          "flickr_url": "xxx",
                          "id": ID
                        }
                ID += 1
                annotations["images"].append(image)
                img2 = copy.deepcopy(img[int(ymin):int(ymax), int(xmin):int(xmax),:])
                cv2.imwrite("./rscup/{}/".format(phase)+filename, img2)
        else:
            filename = imgs[img_id].split(".")[0] + "_{}_{}_".format(str(0), str(0))+"_part" +str(0) + ".jpg"
            image = { "license": 1,
                          "file_name": filename,
                          "coco_url": "xxx",
                          "height": H,
                          "width": W,
                          "date_captured": "2019-06-25",
                          "flickr_url": "xxx",
                          "id": ID
                        }
            ID += 1
            annotations["images"].append(image)
            cv2.imwrite("./rscup/{}/".format(phase)+filename, img)
    with open('./rscup/annotation/annos_rscup_'+phase+'.json', 'w') as json_file:
        json.dump(annotations, json_file, cls=MyEncoder)
        
def generate_anno(filename, height, width, ID):
    image = { "license": 1,
              "file_name": filename,
              "coco_url": "xxx",
              "height": height,
              "width": width,
              "date_captured": "2019-06-25",
              "flickr_url": "xxx",
              "id": ID
            }
    return image
        
def scale_generate_test(datdir, phase, SCALE):
    imgs = os.listdir(datadir)
    rotate90 = iaa.Sequential(
    [iaa.Affine(rotate=90)])
    rotate090 = iaa.Sequential(
    [iaa.Affine(rotate=-90)])
    info = {
        "description": "rscup",
        "url": "http://cocodataset.org",
        "version": "1.0",
        "year": 2014,
        "contributor": "COCO Consortium",
        "date_created": "2017/09/01"
      }
    license = [{ "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/", "id": 1, "name": "Attribution-NonCommercial-ShareAlike License"} ]
    categories = []
    for cls in CLASS:
        category = { "id" : class_to_ind[cls], "name" : cls, "supercategory" : "object",}
        categories.append(category)
    print(categories)
    annotations = {"info": info, "images": [], "annotations": [], "categories":categories, "license":license}
    obj_id = 0
    ID = 0
    for img_id in tqdm(range(len(imgs))):
        img_path = os.path.join(datadir, imgs[img_id])
        original_img = cv2.imread(img_path)
        h, w, _ = original_img.shape
        scales = copy.deepcopy(SCALE)
#         if(max(h, w) > 10000):
#             scales.append(0.1)
        for scale in scales:
            img, scale_factor = mmcv.imrescale(original_img, scale, return_scale=True)
            H,W,_ = img.shape
            if max(H,W) > 512:
                fchips = test_pygenerate(W, H, 512, 416)
                count = 0
                for chip in  fchips:
                    count+=1
                    xmin, ymin, xmax, ymax = chip
                    img2 = copy.deepcopy(img[int(ymin):int(ymax), int(xmin):int(xmax),:])
                    hh, ww, _ = img2.shape
                    filename = imgs[img_id].split(".")[0] + "_{}_{}_{}_{}".format(str(scale), str(xmin), str(ymin), str(0))+"part" +str(count) + ".jpg"
                    cv2.imwrite("./rscup/{}/".format(phase)+filename, img2)
                    image = generate_anno(filename, hh, ww, ID)
                    annotations["images"].append(image)
                    ID += 1
                    
#                     filename = imgs[img_id].split(".")[0] + "_{}_{}_{}_{}".format(str(scale), str(xmin), str(ymin), str(90))+"part" +str(count) + ".jpg"
#                     img_rotate90 = rotate90.augment_image(img2)
#                     cv2.imwrite("./rscup/{}/".format(phase)+filename, img_rotate90)
#                     image = generate_anno(filename, hh, ww, ID)
#                     annotations["images"].append(image)
#                     ID += 1
                    
#                     filename = imgs[img_id].split(".")[0] + "_{}_{}_{}_{}".format(str(scale), str(xmin), str(ymin), str(-90))+"part" +str(count) + ".jpg"
#                     img_rotate090 = rotate090.augment_image(img2)
#                     cv2.imwrite("./rscup/{}/".format(phase)+filename, img_rotate090)
#                     image = generate_anno(filename, hh, ww, ID)
#                     annotations["images"].append(image)
#                     ID += 1     
            else:
                filename = imgs[img_id].split(".")[0] + "_{}_{}_{}_{}".format(str(scale), str(0), str(0), str(0))+"part" +str(0) + ".jpg"
                image = generate_anno(filename, H, W, ID)
                annotations["images"].append(image)
                ID += 1
                cv2.imwrite("./rscup/{}/".format(phase)+filename, img)
                
#                 img_rotate90 = rotate90.augment_image(img)
#                 filename = imgs[img_id].split(".")[0] + "_{}_{}_{}_{}".format(str(scale), str(0), str(0), str(90))+"part" +str(0) + ".jpg"
#                 image = generate_anno(filename, H, W, ID)
#                 annotations["images"].append(image)
#                 ID += 1
#                 cv2.imwrite("./rscup/{}/".format(phase)+filename, img_rotate90)
                
#                 img_rotate090 = rotate090.augment_image(img)
#                 filename = imgs[img_id].split(".")[0] + "_{}_{}_{}_{}".format(str(scale), str(0), str(0), str(-90))+"part" +str(0) + ".jpg"
#                 image = generate_anno(filename,H,W, ID)
#                 annotations["images"].append(image)
#                 ID += 1
#                 cv2.imwrite("./rscup/{}/".format(phase)+filename, img_rotate090)
                
    with open('./rscup/annotation/annos_rscup_'+phase+'.json', 'w') as json_file:
        json.dump(annotations, json_file, cls=MyEncoder)
    print("totol number {}".format(str(ID)))

    
def increment_generate(datdir, phase):
    imgs = os.listdir(datadir)
    coco=COCO('./rscup/annotation/annos_rscup_'+phase+'.json')
    annotations = mmcv.load('./rscup/annotation/annos_rscup_'+phase+'.json')
    imgids = coco.getImgIds()
    ID = max(imgids)+1
    for img_id in tqdm(range(len(imgs))):
        img_path = os.path.join(datadir, imgs[img_id])
        original_img = cv2.imread(img_path)
        h, w, _ = original_img.shape
        scale_factor = 512 /  max(h, w)
        img, scale_factor = mmcv.imrescale(original_img, scale_factor, return_scale=True)
        H,W,_ = img.shape
        filename = imgs[img_id].split(".")[0] + "_{}_{}_{}_{}".format(str(scale_factor), str(0), str(0), str(0))+"part" +str(0) + ".jpg"
        image = { "license": 1,
                          "file_name": filename,
                          "coco_url": "xxx",
                          "height": H,
                          "width": W,
                          "date_captured": "2019-06-25",
                          "flickr_url": "xxx",
                          "id": ID
                        }
        ID += 1
        cv2.imwrite("./rscup/{}/".format(phase)+filename, img)
        annotations["images"].append(image)
        
    with open('./rscup/annotation/annos_rscup_'+phase+'.json', 'w') as json_file:
        json.dump(annotations, json_file, cls=MyEncoder)
    print("totol number {}".format(str(ID)))    
    
    

def generate_nopatch(datdir, labeldir, phase):
    imgs = os.listdir(datadir)
    info = {
        "description": "rscup",
        "url": "http://cocodataset.org",
        "version": "1.0",
        "year": 2014,
        "contributor": "COCO Consortium",
        "date_created": "2017/09/01"
      }
    license = [{ "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/", "id": 1, "name": "Attribution-NonCommercial-ShareAlike License"} ]
    categories = []
    for cls in CLASS:
        category = { "id" : class_to_ind[cls], "name" : cls, "supercategory" : "object",}
        categories.append(category)

    annotations = {"info": info, "images": [], "annotations": [], "categories":categories, "license":license}
    obj_id = 0
    ID = 0
    for img_id in tqdm(range(len(imgs))):
        img_path = os.path.join(datadir, imgs[img_id])
        img = cv2.imread(img_path)
        H,W,_ = img.shape
        label_path = os.path.join(labeldir, imgs[img_id].split(".")[0]+".txt")
        fp = open(label_path).readlines()[2:]
        labels, masks, boxes = get_info(fp)
       
        filename = imgs[img_id]
        image = { "license": 1,
                      "file_name": filename,
                      "coco_url": "xxx",
                      "height": H,
                      "width": W,
                      "date_captured": "2019-06-25",
                      "flickr_url": "xxx",
                      "id": ID
                    }

        #cv2.imwrite("./rscup/{}/".format(phase)+filename, img)
        annotations["images"].append(image)
        annotations, obj_id, ID = gen_ann(boxes, masks, labels, annotations, obj_id, ID,0, 0)
    with open('./data/annotation/annos_rscup_'+phase+'.json', 'w') as json_file:
        json.dump(annotations, json_file, cls=MyEncoder)

        
def get_scale_info(fp, scale):
    masks = []
    labels = []
    boxes = []
    for line in fp:
        line = line.strip()
        if not line:
            continue
        items = line.split(' ')
        loc = np.array([float(x) for x in items[:8]])
        loc = loc*scale
        xs = loc[0::2]
        ys = loc[1::2]
        poly = np.array([[[loc[0],loc[1]],[loc[2],loc[3]],[loc[4],loc[5]],[loc[6], loc[7]]]], np.int32)
        area = cv2.contourArea(poly)
        if(area < 50):
            continue
        xmin  = min(xs)
        xmax = max(xs)
        ymin = min(ys)
        ymax = max(ys)
        w = xmax - xmin
        h = ymax - ymin
        cls = items[8]
        if cls not in CLASS:
            continue
        labels.append(cls)
        masks.append(loc)
        boxes.append([xmin,ymin,xmax,ymax])
    return labels, masks, boxes
        
def scale_generate(datdir, labeldir, phase, SCALE):
    imgs = os.listdir(datadir)
    info = {
        "description": "rscup",
        "url": "http://cocodataset.org",
        "version": "1.0",
        "year": 2014,
        "contributor": "COCO Consortium",
        "date_created": "2017/09/01"
      }
    license = [{ "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/", "id": 1, "name": "Attribution-NonCommercial-ShareAlike License"} ]
    categories = []
    for cls in CLASS:
        category = { "id" : class_to_ind[cls], "name" : cls, "supercategory" : "object",}
        categories.append(category)
    print(categories)
    annotations = {"info": info, "images": [], "annotations": [], "categories":categories, "license":license}
    obj_id = 0
    ID = 0
    for img_id in tqdm(range(len(imgs))):
        img_path = os.path.join(datadir, imgs[img_id])
        label_path = os.path.join(labeldir, imgs[img_id].split(".")[0]+".txt")
        original_img = cv2.imread(img_path)
        h, w, _ = original_img.shape
        scales = copy.deepcopy(SCALE)
#         if(max(h, w) > 10000):
#             scales.append(0.1)
        fp = open(label_path).readlines()[2:]
        for scale in scales:
            img, scale_factor = mmcv.imrescale(original_img, scale, return_scale=True)
            H,W,_ = img.shape
            labels, masks, boxes = get_scale_info(fp, scale_factor)
            if(len(labels)<=0):
                continue
            if max(H,W) > 512:
                fchips, masks, boxes, labels = _pygenerate(boxes, masks, labels, W, H, 512, 416)
                count = 0
                for chip, mask, box, label in zip(fchips, masks, boxes,labels):
                    count+=1
                    xmin, ymin, xmax, ymax = chip
                    filename = imgs[img_id].split(".")[0] + "_{}_{}_{}_{}".format(str(scale), str(xmin), str(ymin), str(0))+"part" +str(count) + ".jpg"
                    img2 = copy.deepcopy(img[int(ymin):int(ymax), int(xmin):int(xmax),:])
                    cv2.imwrite("./rscup/{}/".format(phase)+filename, img2)
                    image = { "license": 1,
                              "file_name": filename,
                              "coco_url": "xxx",
                              "height": ymax-ymin,
                              "width": xmax-xmin,
                              "date_captured": "2019-06-25",
                              "flickr_url": "xxx",
                              "id": ID
                            }
                    annotations["images"].append(image)
                    annotations, obj_id, ID = gen_ann(box, mask, label, annotations, obj_id, ID, xmin, ymin)
            else:
                filename = imgs[img_id].split(".")[0] + "_{}_{}_{}_{}".format(str(scale), str(0), str(0), str(0))+"part" +str(0) + ".jpg"
                image = { "license": 1,
                              "file_name": filename,
                              "coco_url": "xxx",
                              "height": H,
                              "width": W,
                              "date_captured": "2019-06-25",
                              "flickr_url": "xxx",
                              "id": ID
                            }

                cv2.imwrite("./rscup/{}/".format(phase)+filename, img)
                annotations["images"].append(image)
                annotations, obj_id, ID = gen_ann(boxes, masks, labels, annotations, obj_id, ID,0, 0)
    with open('./rscup/annotation/annos_rscup_'+phase+'.json', 'w') as json_file:
        json.dump(annotations, json_file, cls=MyEncoder)
    print("totol number {}".format(str(ID)))
    
    
def increment_train_generate(datdir, labeldir, phase):
    imgs = os.listdir(datadir)
    imgs = os.listdir(datadir)
    coco=COCO('./rscup/annotation/annos_rscup_'+phase+'.json')
    annotations = mmcv.load('./rscup/annotation/annos_rscup_'+phase+'.json')
    imgids = coco.getImgIds()
    annids = coco.getAnnIds()
    ID = max(imgids)+1
    obj_id = max(annids)+1
    for img_id in tqdm(range(len(imgs))):
        img_path = os.path.join(datadir, imgs[img_id])
        label_path = os.path.join(labeldir, imgs[img_id].split(".")[0]+".txt")
        original_img = cv2.imread(img_path)
        h, w, _ = original_img.shape
        scale = 512 /  max(h, w)
#         if(max(h, w) > 10000):
#             scales.append(0.1)
        fp = open(label_path).readlines()[2:]
        img, scale_factor = mmcv.imrescale(original_img, scale, return_scale=True)
        H,W,_ = img.shape
        labels, masks, boxes = get_scale_info(fp, scale_factor)
        if(len(labels)<=0):
            continue
        filename = imgs[img_id].split(".")[0] + "_{}_{}_{}_{}".format(str(scale), str(0), str(0), str(0))+"part" +str(0) + ".jpg"
        image = { "license": 1,
                      "file_name": filename,
                      "coco_url": "xxx",
                      "height": H,
                      "width": W,
                      "date_captured": "2019-06-25",
                      "flickr_url": "xxx",
                      "id": ID
                    }

        cv2.imwrite("./rscup/{}/".format(phase)+filename, img)
        annotations["images"].append(image)
        annotations, obj_id, ID = gen_ann(boxes, masks, labels, annotations, obj_id, ID,0, 0)
    with open('./rscup/annotation/annos_rscup_'+phase+'.json', 'w') as json_file:
        json.dump(annotations, json_file, cls=MyEncoder)
    print("totol number {}".format(str(ID)))

In [ ]:
CLASS=['tennis-court', 'container-crane', 'storage-tank', 'baseball-diamond', 'plane', 'ground-track-field', 'helicopter', 'airport', 'harbor', 'ship', 'large-vehicle', 'swimming-pool', 'soccer-ball-field', 'roundabout', 'basketball-court', 'bridge', 'small-vehicle', 'helipad','noise']
#CLASS={'tennis-court', 'container-crane', 'storage-tank', 'baseball-diamond', 'plane', 'ground-track-field', 'helicopter', 'airport', 'harbor', 'ship', 'large-vehicle', 'swimming-pool', 'soccer-ball-field', 'roundabout', 'basketball-court', 'bridge', 'small-vehicle', 'helipad'}
class_to_ind = dict(zip(CLASS, range(len(CLASS))))
print(class_to_ind)

In [32]:
def bbox_overlaps_py(boxes, query_boxes):
    """
    determine overlaps between boxes and query_boxes
    :param boxes: n * 4 bounding boxes
    :param query_boxes: k * 4 bounding boxes
    :return: overlaps: n * k overlaps
    """
    n_ = boxes.shape[0]
    k_ = query_boxes.shape[0]
    overlaps = np.zeros((n_, k_), dtype=np.float)
    for k in range(k_):
        query_box_area = (query_boxes[k, 2] - query_boxes[k, 0] + 1) * (query_boxes[k, 3] - query_boxes[k, 1] + 1)
        for n in range(n_):
            iw = min(boxes[n, 2], query_boxes[k, 2]) - max(boxes[n, 0], query_boxes[k, 0]) + 1
            if iw > 0:
                ih = min(boxes[n, 3], query_boxes[k, 3]) - max(boxes[n, 1], query_boxes[k, 1]) + 1
                if ih > 0:
                    box_area = (boxes[n, 2] - boxes[n, 0] + 1) * (boxes[n, 3] - boxes[n, 1] + 1)
                    all_area = float(query_box_area)
                    overlaps[n, k] = iw * ih / all_area
    return overlaps

def _pygenerate(boxes, masks, labels, width, height, chipsize, stride):
        chips = []
        boxes = np.array(boxes)
        masks = np.array(masks)
        # ensure coverage of image for worst case
        # corners
        chips.append([max(width - chipsize, 0), 0, width - 1, min(chipsize, height-1)])
        chips.append([0, max(height - chipsize, 0), min(chipsize, width-1), height-1])
        chips.append([max(width - chipsize, 0), max(height - chipsize, 0), width-1, height-1])
		
        for i in range(0, width - int(chipsize), stride):
            for j in range(0, height - int(chipsize), stride):
                x1 = i
                y1 = j
                x2 = i + chipsize - 1
                y2 = j + chipsize - 1
                chips.append([x1, y1, x2, y2])
		#width may not be divide by stride
        for j in range(0, height - int(chipsize), stride):
            x1 = max(width - chipsize - 1,0)
            y1 = j
            x2 = width - 1
            y2 = j + chipsize - 1
            chips.append([x1, y1, x2, y2])
		#the same as above
        for i in range(0, width - int(chipsize), stride):
            x1 = i
            y1 = max(height - chipsize - 1,0)
            x2 = i + chipsize - 1
            y2 = height - 1
            chips.append([x1, y1, x2, y2])

        chips = np.array(chips).astype(np.float)
        overlaps = bbox_overlaps_py(chips, boxes.astype(np.float))
        fchips = []
        masks_list = []
        boxes_list = []
        labels_list = []
        for j in range(len(chips)):
            nvids = np.where(overlaps[j, :] >= 0.9)[0]
            if(len(nvids) == 0):
                continue
            else:
                fchips.append(chips[j])
                boxes_list.append(boxes[nvids])
                masks_list.append(masks[nvids])
                labels_list.append([labels[x] for x in nvids])
        return fchips, masks_list, boxes_list, labels_list
def get_scale_info(fp, scale):
    masks = []
    labels = []
    boxes = []
    for line in fp:
        line = line.strip()
        if not line:
            continue
        items = line.split(' ')
        loc = np.array([float(x) for x in items[:8]])
        loc = loc*scale
        xs = loc[0::2]
        ys = loc[1::2]
        poly = np.array([[[loc[0],loc[1]],[loc[2],loc[3]],[loc[4],loc[5]],[loc[6], loc[7]]]], np.int32)
        area = cv2.contourArea(poly)
        if(area < 50):
            continue
        xmin  = min(xs)
        xmax = max(xs)
        ymin = min(ys)
        ymax = max(ys)
        w = xmax - xmin
        h = ymax - ymin
        cls = items[8]
        if cls not in CLASS:
            continue
        labels.append(cls)
        masks.append(loc)
        boxes.append([xmin,ymin,xmax,ymax])
    return labels, masks, boxes
def construct_imginfo(filename, h, w, ID):
    image = { "license": 1,
              "file_name": filename,
              "coco_url": "xxx",
              "height": h,
              "width": w,
              "date_captured": "2019-06-25",
              "flickr_url": "xxx",
              "id": ID
            }
    return image

def construct_ann(obj_id, ID, category_id, seg, area, bbox):
    ann = {"id" : obj_id, 
                "image_id" : ID, 
                "category_id" : category_id, 
                "segmentation" : seg, 
               "area" :  area, 
               "bbox" : bbox, 
               "iscrowd" : 0,
            }
    return ann

def generate_ann(box, mask, label, xmin, ymin):
    ann = []
    for b, loc, l in zip(box, mask, label):
        x1, y1, x2, y2 = b
        x1 -= xmin
        y1 -= ymin
        x2 -= xmin
        y2 -= ymin
        poly = np.array([[[loc[0],loc[1]],[loc[2],loc[3]],[loc[4],loc[5]],[loc[6], loc[7]]]], np.int32)
        box = [x1, y1, x2, y2]
        area = cv2.contourArea(poly)
        poly = [loc[0]-xmin,loc[1]-ymin,loc[2]-xmin,loc[3]-ymin,loc[4]-xmin,loc[5]-ymin,loc[6]-xmin, loc[7]-ymin]
        obj = [class_to_ind[l], [poly], area, [x1, y1, x2-x1, y2-y1]]
        ann.append(obj)
    return ann

def chip_warpper(mode, chip, imgname, img, box, mask, label, scale, outdir, phase="train", count=0):
    if mode == "multi":
        xmin, ymin, xmax, ymax = chip
        filename = imgname.split(".")[0] + "_{}_{}_{}_{}".format(str(scale), str(xmin), str(ymin), str(0))+"part" +str(count) + ".jpg"
        cv2.imwrite("{}/{}".format(outdir, phase)+filename, img)
        h, w,_ = img.shape
        ann = generate_ann(box, mask, label, xmin, ymin)
    else:
        filename = imgname.split(".")[0] + "_{}_{}_{}_{}".format(str(scale), str(0), str(0), str(0))+"part" +str(0) + ".jpg"
        h, w,_ = img.shape
        cv2.imwrite("{}/{}".format(outdir, phase)+filename, img)
        ann = generate_ann(box, mask, label, 0, 0)
    return (ann, filename, h, w)
   

def scale_generate(datdir, labeldir, phase, outdir, SCALE):
    imgs = os.listdir(datadir)
    info = {
        "description": "rscup",
        "url": "http://cocodataset.org",
        "version": "1.0",
        "year": 2014,
        "contributor": "COCO Consortium",
        "date_created": "2017/09/01"
      }
    license = [{ "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/", "id": 1, "name": "Attribution-NonCommercial-ShareAlike License"} ]
    categories = []
    for cls in CLASS:
        category = { "id" : class_to_ind[cls], "name" : cls, "supercategory" : "object",}
        categories.append(category)
    print(categories)
    annotations = {"info": info, "images": [], "annotations": [], "categories":categories, "license":license}
    obj_id = 0
    ID = 0
    p = Pool(18)
    for img_id in tqdm(range(len(imgs))):
        img_path = os.path.join(datadir, imgs[img_id])
        label_path = os.path.join(labeldir, imgs[img_id].split(".")[0]+".txt")
        original_img = cv2.imread(img_path)
        scales = copy.deepcopy(SCALE)
        fp = open(label_path).readlines()[2:]
        rets = []
        for scale in scales:
            img, scale_factor = mmcv.imrescale(original_img, scale, return_scale=True)
            H,W,_ = img.shape
            labels, masks, boxes = get_scale_info(fp, scale_factor)
            if(len(labels)<=0):
                continue
            if max(H,W) > 512:
                print(np.array(masks).shape)
                print(len(labels))
                fchips, masks, boxes, labels = chips.generate(boxes, masks, labels, W, H, 512, 416)
                count = 0
                for chip, mask, box, label in zip(fchips, masks, boxes,labels):
                    count+=1
                    xmin, ymin, xmax, ymax = chip
                    img2 = copy.deepcopy(img[int(ymin):int(ymax), int(xmin):int(xmax),:])
                    rets.append(p.apply_async(chip_warpper, args=("multi", chip, imgs[img_id], img2, box, mask, label, scale, outdir, count)))
            else:
                rets.append(p.apply_async(chip_warpper, args=("single", None, imgs[img_id], img, boxes, masks, labels, scale, outdir, count)))
        for ret in rets:
            anns, filename, h, w = ret.get()
            image = construct_imginfo(filename, h, w, ID)
            annotations["images"].append(image)
            for ann in anns:
                category_id, seg, area, bbox = ann
                ann_info = construct_ann(obj_id, ID, category_id, seg, area, bbox)
                annotations["annotations"].append(ann_info)
                obj_id += 1
            ID += 1
    p.close()
    p.join()
    with open('{}/annos_rscup_'.format(outdir)+phase+'.json', 'w') as json_file:
        json.dump(annotations, json_file, cls=MyEncoder)
    print("totol number {}".format(str(ID)))

In [33]:
CLASS=['tennis-court', 'container-crane', 'storage-tank', 'baseball-diamond', 'plane', 'ground-track-field', 'helicopter', 'airport', 'harbor', 'ship', 'large-vehicle', 'swimming-pool', 'soccer-ball-field', 'roundabout', 'basketball-court', 'bridge', 'small-vehicle', 'helipad']
#CLASS={'tennis-court', 'container-crane', 'storage-tank', 'baseball-diamond', 'plane', 'ground-track-field', 'helicopter', 'airport', 'harbor', 'ship', 'large-vehicle', 'swimming-pool', 'soccer-ball-field', 'roundabout', 'basketball-court', 'bridge', 'small-vehicle', 'helipad'}
class_to_ind = dict(zip(CLASS, range(len(CLASS))))
print(class_to_ind)
datadir = "/home/xfr/rssid/data/train/images"
labeldir = "/home/xfr/rssid/data/train/labelTxt"
scale_generate(datadir, labeldir, "train","./data/trash", [1])

{'tennis-court': 0, 'container-crane': 1, 'storage-tank': 2, 'baseball-diamond': 3, 'plane': 4, 'ground-track-field': 5, 'helicopter': 6, 'airport': 7, 'harbor': 8, 'ship': 9, 'large-vehicle': 10, 'swimming-pool': 11, 'soccer-ball-field': 12, 'roundabout': 13, 'basketball-court': 14, 'bridge': 15, 'small-vehicle': 16, 'helipad': 17}
[{'id': 0, 'name': 'tennis-court', 'supercategory': 'object'}, {'id': 1, 'name': 'container-crane', 'supercategory': 'object'}, {'id': 2, 'name': 'storage-tank', 'supercategory': 'object'}, {'id': 3, 'name': 'baseball-diamond', 'supercategory': 'object'}, {'id': 4, 'name': 'plane', 'supercategory': 'object'}, {'id': 5, 'name': 'ground-track-field', 'supercategory': 'object'}, {'id': 6, 'name': 'helicopter', 'supercategory': 'object'}, {'id': 7, 'name': 'airport', 'supercategory': 'object'}, {'id': 8, 'name': 'harbor', 'supercategory': 'object'}, {'id': 9, 'name': 'ship', 'supercategory': 'object'}, {'id': 10, 'name': 'large-vehicle', 'supercategory': 'objec

(582, 8)
582


TypeError: generate() takes exactly 5 positional arguments (7 given)

In [28]:
import chips.chips as chips